In [1]:
import sys
sys.path.append('../cgm_database/')
import dbutils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandasql as ps
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt
import scipy
import chart_studio.plotly as py
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [2]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [3]:
main_connector = dbutils.connect_to_main_database('../../prod_dbconnection.json')

Loading /mnt/batch/tasks/shared/LS_root/mounts/clusters/prajwal-4cpu/code/users/psingh/prod_dbconnection.json...
Host:    cgm-in-bmz-postgresql-sql-prod.postgres.database.azure.com
DB-name: cgm


In [4]:
#select_personlab="SELECT m_truth.age,ar_height.model_id,abs(m_truth.height - ar_height.float_value) AS ae, ar_pose.json_value->'pose_1' -> 'nose'->'score' as nose,ar_pose.json_value->'pose_1' -> 'leftAnkle'->'score',ar_pose.json_value->'pose_1' -> 'leftEar'->'score',ar_pose.json_value->'pose_1' -> 'leftElbow'->'score',ar_pose.json_value->'pose_1' -> 'leftEye'->'score',ar_pose.json_value->'pose_1' -> 'leftHip'->'score',ar_pose.json_value->'pose_1' -> 'leftKnee'->'score',ar_pose.json_value->'pose_1' -> 'leftShoulder'->'score',ar_pose.json_value->'pose_1' -> 'leftWrist'->'score',ar_pose.json_value->'pose_1' -> 'rightAnkle'->'score',ar_pose.json_value->'pose_1' -> 'rightEar'->'score',ar_pose.json_value->'pose_1' -> 'rightElbow'->'score',ar_pose.json_value->'pose_1' -> 'rightEye'->'score',ar_pose.json_value->'pose_1' -> 'rightHip'->'score',ar_pose.json_value->'pose_1' -> 'rightKnee'->'score',ar_pose.json_value->'pose_1' -> 'rightShoulder'->'score',ar_pose.json_value->'pose_1' -> 'rightWrist'->'score' FROM measure m_scan JOIN measure m_truth ON m_scan.qr_code::text = m_truth.qr_code::text AND m_scan.age = m_truth.age JOIN artifact a_pcd ON a_pcd.measure_id = m_scan.id::text JOIN artifact_result ar_height ON ar_height.artifact_id::text = a_pcd.id::text JOIN artifact a_rgb ON a_pcd.measure_id = a_rgb.measure_id AND round(a_pcd.session_timestamp::numeric, 2) = round(a_rgb.session_timestamp::numeric, 2) JOIN artifact_result ar_pose ON ar_pose.artifact_id::text = a_rgb.id::text WHERE a_pcd.dataformat = 'pcd'::text AND a_rgb.dataformat = 'rgb'::text AND m_truth.type = 'manual'::text AND ar_pose.model_id ~~ '%pose%'::text AND ar_height.model_id like '%height%' AND ar_pose.float_value=1";
#artifacts = pd.DataFrame(main_connector.execute(select_personlab, fetch_all=True), columns=['age','model_id','error','json_value_pose_1_nose_score','json_value_pose_1_leftAnkle_score','json_value_pose_1_leftEar_score','json_value_pose_1_leftElbow_score','json_value_pose_1_leftEye_score','json_value_pose_1_leftHip_score','json_value_pose_1_leftKnee_score','json_value_pose_1_leftShoulder_score','json_value_pose_1_leftWrist_score','json_value_pose_1_rightAnkle_score','json_value_pose_1_rightEar_score','json_value_pose_1_rightElbow_score','json_value_pose_1_rightEye_score','json_value_pose_1_rightHip_score','json_value_pose_1_rightKnee_score','json_value_pose_1_rightShoulder_score','json_value_pose_1_rightWrist_score'])


In [5]:
#artifacts.to_csv('../../posentvsheightmodel.csv',index=false)

In [21]:
artifacts=pd.read_csv('../../posentvsheightmodel.csv').drop(['Unnamed: 0'],axis=1)

In [24]:
artifacts.shape

(3715, 20)

In [8]:
artifacts=artifacts[artifacts['age']<=1825]
artifacts=artifacts[artifacts['age']>0]

In [9]:
artifacts['age'].iplot(kind='hist', xTitle='Age',
                  yTitle='count', title='Age Distribution')

In [10]:
model_id=artifacts['model_id'].unique().tolist()

In [11]:
@interact_manual
def show_articles_more_than(age_start=(0,1825,180),model_id=model_id,x=list(artifacts.select_dtypes('number').columns[2:],), 
                 y=list(artifacts.select_dtypes('number').columns)[1:2],error=[0.2,0.4,0.6,0.8,1.0,1.2,1.5,2,2.5,3.0,3.5,4.0,10.0,20.0,40,60]):
    dff=artifacts.loc[artifacts['model_id']==model_id]
    dff=dff.loc[dff['age'] > age_start]
    dff=dff.loc[dff['error'] < error]
    xaxis=dff[x].tolist()
    yaxis=dff[y].tolist()
    print(len(xaxis))
    print("Cofficient constant R",scipy.stats.pearsonr(xaxis, yaxis)[0])
    dff.iplot(kind='scatter',x=x,y=y,mode='markers')

interactive(children=(IntSlider(value=900, description='age_start', max=1825, step=180), Dropdown(description=…

In [12]:
#select_personlab="SELECT m_truth.age,ar_height.model_id,abs(m_truth.height - ar_height.float_value) AS ae, ar_pose.json_value->'pose_1' -> 'nose'->'score' as nose,ar_pose.json_value->'pose_1' -> 'leftAnkle'->'score',ar_pose.json_value->'pose_1' -> 'leftEar'->'score',ar_pose.json_value->'pose_1' -> 'leftElbow'->'score',ar_pose.json_value->'pose_1' -> 'leftEye'->'score',ar_pose.json_value->'pose_1' -> 'leftHip'->'score',ar_pose.json_value->'pose_1' -> 'leftKnee'->'score',ar_pose.json_value->'pose_1' -> 'leftShoulder'->'score',ar_pose.json_value->'pose_1' -> 'leftWrist'->'score',ar_pose.json_value->'pose_1' -> 'rightAnkle'->'score',ar_pose.json_value->'pose_1' -> 'rightEar'->'score',ar_pose.json_value->'pose_1' -> 'rightElbow'->'score',ar_pose.json_value->'pose_1' -> 'rightEye'->'score',ar_pose.json_value->'pose_1' -> 'rightHip'->'score',ar_pose.json_value->'pose_1' -> 'rightKnee'->'score',ar_pose.json_value->'pose_1' -> 'rightShoulder'->'score',ar_pose.json_value->'pose_1' -> 'rightWrist'->'score' FROM measure m_scan JOIN measure m_truth ON m_scan.qr_code::text = m_truth.qr_code::text AND m_scan.age = m_truth.age JOIN artifact a_pcd ON a_pcd.measure_id = m_scan.id::text JOIN artifact_result ar_height ON ar_height.artifact_id::text = a_pcd.id::text JOIN artifact a_rgb ON a_pcd.measure_id = a_rgb.measure_id AND round(a_pcd.session_timestamp::numeric, 2) = round(a_rgb.session_timestamp::numeric, 2) JOIN artifact_result ar_pose ON ar_pose.artifact_id::text = a_rgb.id::text WHERE a_pcd.dataformat = 'pcd'::text AND a_rgb.dataformat = 'rgb'::text AND m_truth.type = 'manual'::text AND ar_pose.model_id ~~ '%person%'::text AND ar_height.model_id like '%height%' AND ar_pose.float_value=1";
#personlab_artifact = pd.DataFrame(main_connector.execute(select_personlab, fetch_all=True), columns=['age','model_id','error','json_value_pose_1_nose_score','json_value_pose_1_leftAnkle_score','json_value_pose_1_leftEar_score','json_value_pose_1_leftElbow_score','json_value_pose_1_leftEye_score','json_value_pose_1_leftHip_score','json_value_pose_1_leftKnee_score','json_value_pose_1_leftShoulder_score','json_value_pose_1_leftWrist_score','json_value_pose_1_rightAnkle_score','json_value_pose_1_rightEar_score','json_value_pose_1_rightElbow_score','json_value_pose_1_rightEye_score','json_value_pose_1_rightHip_score','json_value_pose_1_rightKnee_score','json_value_pose_1_rightShoulder_score','json_value_pose_1_rightWrist_score'])


In [13]:
#personlab_artifact.to_csv('personlabvsheight')

In [25]:
personlab_artifact=pd.read_csv('../../personlabvsheight').drop(['Unnamed: 0'],axis=1)

In [27]:
personlab_artifact=personlab_artifact[personlab_artifact['age']<=1825]
personlab_artifact=personlab_artifact[personlab_artifact['age']>0]

In [28]:
personlab_artifact['age'].iplot(kind='hist', xTitle='Age',
                  yTitle='count', title='Age Distribution')

In [29]:
#add age
@interact_manual
def show_articles_more_than(age_start=(0,1825,180),model_id=model_id,x=list(personlab_artifact.select_dtypes('number').columns[2:],), 
                 y=list(personlab_artifact.select_dtypes('number').columns)[1:2],error=[0.2,0.4,0.6,0.8,1.0,1.2,1.5,2,2.5,3.0,3.5,4.0,10.0,20.0,40,60]):
    dff=personlab_artifact.loc[personlab_artifact['model_id']==model_id]
    dff=dff.loc[dff['age'] > age_start]
    dff=dff.loc[dff['error'] < error]
    xaxis=dff[x].tolist()
    yaxis=dff[y].tolist()
    print(len(xaxis))
    print("Cofficient constant R",scipy.stats.pearsonr(xaxis, yaxis)[0])
    dff.iplot(kind='scatter',x=x,y=y,mode='markers')

interactive(children=(IntSlider(value=900, description='age_start', max=1825, step=180), Dropdown(description=…